In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import visual_behavior.visualization.utils as utils
import visual_behavior.data_access.loading as loading
import visual_behavior.data_access.filtering as filtering
import visual_behavior.ophys.response_analysis.utilities as ut
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

### get all experiments, including failed ones

In [4]:
experiments_table = loading.get_filtered_ophys_experiment_table(include_failed_data=True) 

In [5]:
len(experiments_table)

2260

### what are we willing to un-fail? 

* d_prime_peak
* num_contingent_trials
* eye_tracking_global_mean = can't see eye or animal stress

#### what to do with these fail causes? 


* validate_omitted_flashes_are_omitted
* validate_one_stimulus_per_aborted_trial
* validate_session_within_expected_duration
* validate_two_stimuli_per_go_trial
* validate_licks
* parent_vasculature_image
* wheel_speed_plot
* validate_initial_blank
* validate_stage_present

### how many experiments have these fail causes? 

* d_prime_peak
* num_contingent_trials
* eye_tracking_global_mean

In [6]:
experiments_table['fail_tags'] = [tag.split(',') if type(tag)==str else tag for tag in experiments_table.failure_tags.values]

In [7]:
len(experiments_table)

2260

In [8]:
tags_to_unfail = ['d_prime_peak', 'num_contingent_trials', 'eye_tracking_global_mean']

In [10]:
unfail_list = []
for row in range(len(experiments_table)):
    row_data = experiments_table.iloc[row]
    if type(row_data.fail_tags)==list:
        for fail_tag in tags_to_unfail: 
            if (fail_tag in row_data.fail_tags) and (len(row_data.fail_tags)==1):
                unfail_list.append([row_data.name, fail_tag])
unfail_df = pd.DataFrame(data=unfail_list, columns=['ophys_experiment_id', 'fail_tag'])
unfail_df

,ophys_experiment_id,fail_tag
0,863735602,d_prime_peak
1,1012468625,d_prime_peak
2,918566488,d_prime_peak
3,1061787725,d_prime_peak
4,1062828011,d_prime_peak


In [184]:
passed_expts = loading.get_filtered_ophys_experiment_table(include_failed_data=False) 
print(len(unfail_df),'expts out of', len(passed_expts), '=', np.round(float(len(unfail_df))/float(len(passed_expts)), 3),
     'percent of total dataset could be unfailed for these reasons')

159 expts out of 1638 = 0.097 percent of total dataset could be unfailed for these reasons


In [182]:
df = unfail_df.merge(experiments_table, on='ophys_experiment_id')

In [183]:
for cre_line in df.cre_line.unique():
    print(len(df[df.cre_line==cre_line]), cre_line)
print('')
for project_code in df.project_code.unique():
    print(len(df[df.project_code==project_code]), project_code)
print('')
print(len(df[(df.session_number==4)&(df.prior_exposures_to_image_set==0)]), 'novel not retakes')

95 Slc17a7-IRES2-Cre
44 Sst-IRES-Cre
20 Vip-IRES-Cre

46 VisualBehavior
51 VisualBehaviorTask1B
62 VisualBehaviorMultiscope

35 novel not retakes


In [11]:
experiments_table

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,...,failure_tags,stimulus_type_exposure_number,prior_exposures_to_image_set,prior_exposures_to_omissions,model_outputs_available,has_cell_matching,location,has_events,session_number,fail_tags
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
953659743,952430817,952554548,1018028345,VisualBehaviorMultiscope,container_qc,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,...,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,65.0,0.0,False,True,Sst_VISp_225,True,2.0,"[z_drift_corr_um_diff, z_drift_corr_um_diff, z..."
953659745,952430817,952554548,1018028339,VisualBehaviorMultiscope,container_qc,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,...,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,65.0,0.0,False,True,Sst_VISp_75,True,2.0,"[z_drift_corr_um_diff, z_drift_corr_um_diff, z..."
953659749,952430817,952554548,1018028354,VisualBehaviorMultiscope,container_qc,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,...,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,65.0,0.0,False,True,Sst_VISl_150,True,2.0,"[z_drift_corr_um_diff, z_drift_corr_um_diff, z..."
953659752,952430817,952554548,1018028357,VisualBehaviorMultiscope,container_qc,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,...,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,65.0,0.0,False,True,Sst_VISl_225,True,2.0,"[z_drift_corr_um_diff, z_drift_corr_um_diff, z..."
958527474,954954402,953982960,1018028339,VisualBehaviorMultiscope,container_qc,passed,20190924_457841_3imagesA,OPHYS_3_images_A,MESO.1,2019-09-24 16:00:00.000000,...,NaN,0,66.0,1.0,True,True,Sst_VISp_75,True,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929655737,929255311,929341697,1026722867,VisualBehaviorMultiscope,postprocessing,passed,20190822_453988_1imagesA,OPHYS_1_images_A,MESO.1,2019-08-22 08:21:33.000000,...,NaN,0,42.0,20.0,True,False,Slc17a7_VISl_85,True,1.0,NaN
929655732,929255311,929341697,1026722870,VisualBehaviorMultiscope,container_qc,passed,20190822_453988_1imagesA,OPHYS_1_images_A,MESO.1,2019-08-22 08:21:33.000000,...,NaN,0,42.0,20.0,True,True,Slc17a7_VISl_170,True,1.0,NaN
929655728,929255311,929341697,1026722854,VisualBehaviorMultiscope,postprocessing,passed,20190822_453988_1imagesA,OPHYS_1_images_A,MESO.1,2019-08-22 08:21:33.000000,...,NaN,0,42.0,20.0,True,False,Slc17a7_VISp_55,False,1.0,NaN
